# Tic tac toe

In [1]:
import random
class TictacToe:
    def __init__(self,player1,player2):
        self.player_1 = player1
        self.player_2 = player2
        self.reset()
    
    def reset(self, ):
        self.board = [" "] * 9 #9 blank place in a list formate
        self.player_1_turn = random.choice([True,False]) #suru ma jun player ley pani palo pauna sak6 so randomly jasko palo ni auna sak6
        
    def check_full(self, ):
        emptys = [x for x in self.board if x == ' '] # board ko sabai box ma ja input x if the box is empty
        if len(emptys) == 0:
            return True
        return False
    
    def print_board(self):
        b = self.board
        #board design
        board = f"""
        {b[0]} | {b[1]} | {b[2]}
        ---------
        {b[3]} | {b[4]} | {b[5]}
        ---------
        {b[6]} | {b[7]} | {b[8]}
        """
        print(board)
        
    def check_winner(self,char):
        b = self.board
        winner = False
        #lookup table banauna lako
        conditions = [
            (0,1,2),
            (3,4,5),
            (6,7,8),
            (0,3,6),
            (1,4,7),
            (2,5,8),
            (0,4,8),
            (2,4,6)
        ]
        for each_condition in conditions:
            i1,i2,i3 = each_condition
            if char == b[i1] == b[i2] == b[i3]:
                winner = True
                break
        return winner
    def play(self, ):
        self.player_1.new_game()
        self.player_2.new_game()
        self.reset()
        
        while True:
            if self.player_1_turn:
                player = self.player_1
                other_player =self.player_2
            else:
                player = self.player_2
                other_player =self.player_2
            
            #Ask for move
            if player.type =='human':
                print(f"{player.name}'s Turn")
                self.print_board()
            move = player.make_move()
            
            #check validity
            if move <0 or move > 8:
                print('Invalid Move')
                player.reward(-0.25,self.board)
                continue
            if self.board[move] != ' ':
                print('Invalid Move')
                continue
                
            #put char
            self.board[move] = player.char
            
            #check winner
            win = self.check_winner(player.char)
            if win:
                player.winner()
                player.reward(5,self.board)
                other_player.reward(-5,self.board)
                self.print_board()
                break
                
            #check full or Draw vako hoki haina herna lako
            full = self.check_full()
            if full:
                print('Game Ended as Draw')
                player.reward('Orange Ball')
                break
            player.reward(0.25,self.board)
            #swapping the turn
            self.player_1_turn = not self.player_1_turn
            

In [2]:
#player ko class
class Player:
    def __init__(self, name,char,type = 'human'):
        self.name = name
        self.char = char
        self.type = type

    def make_move(self):#user bata input liney lai 
        ans = input('Enter your move: ')
        return int(ans)
    
    def  new_game(self):
        print(f'{self.name} in {self.char}')
        
    def winner(self):
        print(f'{self.name} is winner')
        
    def reward(self,reward_value):
        print(f'{self.name} gets reward of {reward_value}')
    def available_moves(self,board):
        return [x for x in range(9) if board[x]=='']

In [3]:
p1 = Player('Hari Bahadur', 'X')
p2 = Player('Timila', 'O')

In [4]:
#unit testing
p1.reward(10)

Hari Bahadur gets reward of 10


In [5]:
#unit testing
p2.winner()

Timila is winner


In [6]:
#unit testing
p2.new_game()

Timila in O


In [7]:
p2.make_move()

Enter your move: 5


5

In [8]:
game =TictacToe(p1,p2)

In [9]:
#user testing
game.print_board()


          |   |  
        ---------
          |   |  
        ---------
          |   |  
        


In [10]:
game.check_full()

False

In [11]:
game.play()

Hari Bahadur in X
Timila in O
Hari Bahadur's Turn

          |   |  
        ---------
          |   |  
        ---------
          |   |  
        
Enter your move: 8'


ValueError: invalid literal for int() with base 10: "8'"

In [ ]:
class AI(Player):
    def __init__(self,name,char,epsilon,gamma,alpha):
        self.name=name
        self.char=char
        self.type_='AI'
        self.epsilon=epsilon  #Epsilon-Greedy
        self.gamma=gamma      #Discount Factor
        self.alpha=alpha      #learning Rate
        self.q_table={}
    def new_game(self):
        self.prev_board=()
        self.prev_action=None
    def winner(self):
        pass
    def get_Q(self,state,action):
        if self.q_table.get((state,action))==None:
            self.q_table[(state,action)]=5
        return self.q_table[(state,action)]
    def make_move(self,board):
        #save old state
        self.prev_board=tuple(board)
        available_action=self.available_moves(board)
        #Eplison Greedy method
        if random.random()<self.epsilon:
            action_taken=random.choice(available_action)
            self.prev_action=action_taken
            self.epsilon*=0.9999
            return action_taken
        q_values=[]
        for action in available_action:
            q_values.append(self.get_Q(self.prev_board,action))
        max_q_val=max(q_values)
        index=q_values.index(max_q_val)
        
        action_taken = available_action[index]
        self.prev_action=action_taken
        return action_taken
    def reward(self,value,board):
        if self.prev_action:
            self.q_learn(self.prev_board,self.prev_action,value, tuple(board))
        
    
    def q_learn(self,state,action,reward,new_state):
        prev_q_val = self.get_Q(state,action)
        available_action= self.available_moves(state)
        q_values=[self.get_Q(new_state, action)for action in available_action ]
        max_q=max(q_values)
        self.q_learn[(state,action)]=prev_q_val + self.alpha * (reward + sel.gamma*(max_q)-prev_q_val)
        